In [1]:
import numpy as np
import argparse
import sys
import cv2
import time
from math import pow, sqrt
from imutils.video import FPS

# Parse the arguments from command line
parser = argparse.ArgumentParser()

parser.add_argument('-v', '--video', type = str, default = 'demo.mp4', help = 'Video file path. If no path is given, video is captured using device.')

parser.add_argument('-m', '--model', default = 'SSD_MobileNet.caffemodel', help = "Path to the pretrained model.")
    
parser.add_argument('-p', '--prototxt', default = 'SSD_MobileNet_prototxt.txt', help = 'Prototxt of the model.')

parser.add_argument('-l', '--labels', default = 'class_labels.txt', help = 'Labels of the dataset.')

parser.add_argument('-c', '--confidence', type = float, default = 0.8, help='Set confidence for detecting objects')

args = parser.parse_args(args=[])


labels  = ["background", "aeroplane", "bicycle", "bird", "boat","bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
            "dog", "horse", "motorbike", "person", "pottedplant", "sheep","sofa", "train", "tvmonitor"]


# Generate random bounding_box_color for each label
COLORS = np.random.uniform(0, 255, size=(len(labels), 3))


# Load model
print("\nLoading model...\n")
network = cv2.dnn.readNetFromCaffe(args.prototxt, args.model)

print("\nStreaming video using device...\n")


# Capture video from file or through device
if args.video:
    cap = cv2.VideoCapture(args.video)
else:
    cap = cv2.VideoCapture(0)
time.sleep(2.0)
fps = FPS().start()
frame_no = 0

while cap.isOpened():

    frame_no = frame_no+1

    # Capture one frame after another
    ret, frame = cap.read()

    if not ret:
        break

    (h, w) = frame.shape[:2]

    # Resize the frame to suite the model requirements. Resize the frame to 300X300 pixels
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    network.setInput(blob)
    detections = network.forward()
    
    # loop over the detections
    for i in np.arange(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with the prediction
        confidence = detections[0, 0, i, 2]
        # filter out weak detections by ensuring the `confidence` is greater than the minimum confidence
        if confidence > args.confidence:
            # extract the index of the class label from the`detections`, then compute the (x, y)-coordinates of
            # the bounding box for the object
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            # draw the prediction on the frame
            label = "{}: {:.2f}%".format(labels[idx],confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY),COLORS[idx], 2)
            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y-20), cv2.FONT_HERSHEY_DUPLEX, 0.5, COLORS[idx], 2)

    pos_dict = dict()
    coordinates = dict()

    # Focal length (in millimeters)
    F = 50

    for i in range(detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > args.confidence:

            class_id = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            coordinates[i] = (startX, startY, endX, endY)

            # Mid point of bounding box
            x_mid = round((startX+endX)/2,4)
            y_mid = round((startY+endY)/2,4)

            height = round(endY-startY,4)

            # Distance from camera based on triangle similarity
            distance = (165 * F)/height
            print("Distance(cms):{dist}\n".format(dist=distance))

            # Mid-point of bounding boxes (in cm) based on triangle similarity technique
            x_mid_cm = (x_mid * distance) / F
            y_mid_cm = (y_mid * distance) / F
            pos_dict[i] = (x_mid_cm,y_mid_cm,distance)

    # Distance between every object detected in a frame
    close_objects = set()
    for i in pos_dict.keys():
        for j in pos_dict.keys():
            if i < j:
                dist = sqrt(pow(pos_dict[i][0]-pos_dict[j][0],2) + pow(pos_dict[i][1]-pos_dict[j][1],2) + pow(pos_dict[i][2]-pos_dict[j][2],2))

                # Check if distance less than 1 foot (300 mm approx):
                if dist < 300:
                    close_objects.add(i)
                    close_objects.add(j)

    for i in pos_dict.keys():
        if i in close_objects:
            COLOR = (0,0,255)
        else:
            COLOR = (0,255,0)
     
        (startX, startY, endX, endY) = coordinates[i]

        cv2.rectangle(frame,(startX,startY), (endX, endY), COLOR, 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        # Convert mms to feet
        cv2.putText(frame, 'Distance: {i} ft'.format(i=round(pos_dict[i][2]/30.48,4)), (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR, 2)

    cv2.namedWindow('Frame',cv2.WINDOW_NORMAL)

    # Show frame
    cv2.imshow('Frame', frame)
    cv2.resizeWindow('Frame',1400,1200)

    key = cv2.waitKey(1) & 0xFF

    # Press `q` to exit
    if key == ord("q"):
        break
    # update the FPS counter
    fps.update()
# stop the timer and display FPS information
fps.stop()
print("Elapsed time: {:.2f}".format(fps.elapsed()))
print("FPS: {:.2f}".format(fps.fps()))
# Clean
cap.release()
cv2.destroyAllWindows()


Loading model...


Streaming video using device...

Distance(cms):4.323899371069182

Distance(cms):4.44264943457189

Distance(cms):4.547960308710033

Distance(cms):4.425965665236052

Distance(cms):4.192073170731708

Distance(cms):4.537953795379538

Distance(cms):4.476397178513293

Distance(cms):4.568106312292358

Distance(cms):4.228600717580727

Distance(cms):4.285714285714286

Distance(cms):4.423592493297587

Distance(cms):4.580788450860633

Distance(cms):4.319371727748691

Distance(cms):4.381306425916091

Distance(cms):4.585881045025014

Distance(cms):4.296875

Distance(cms):4.596100278551532

Distance(cms):4.445043103448276

Distance(cms):4.585881045025014

Distance(cms):4.449838187702265

Distance(cms):4.270186335403727

Distance(cms):4.272397721387882

Distance(cms):4.621848739495798

Distance(cms):4.425965665236052

Distance(cms):4.250386398763524

Distance(cms):4.59866220735786

Distance(cms):4.332983193277311

Distance(cms):5.149812734082397

Distance(cms):4.570637119113574

D

Distance(cms):5.268199233716475

Distance(cms):4.910714285714286

Distance(cms):5.1020408163265305

Distance(cms):4.104477611940299

Distance(cms):5.248091603053435

Distance(cms):5.201765447667087

Distance(cms):4.937163375224417

Distance(cms):4.064039408866995

Distance(cms):4.966887417218543

Distance(cms):5.261479591836735

Distance(cms):5.175658720200753

Distance(cms):4.072063178677197

Distance(cms):5.264837268666241

Distance(cms):4.9490101979604075

Distance(cms):5.108359133126935

Distance(cms):3.9398280802292263

Distance(cms):4.910714285714286

Distance(cms):5.1020408163265305

Distance(cms):5.364109232769831

Distance(cms):3.921102661596958

Distance(cms):3.90625

Distance(cms):4.896142433234421

Distance(cms):5.064456721915286

Distance(cms):5.3260167850225955

Distance(cms):3.949258018190522

Distance(cms):4.847238542890717

Distance(cms):4.99092558983666

Distance(cms):5.305466237942122

Distance(cms):3.9360687022900764

Distance(cms):4.946043165467626

Distance(cms):5

Distance(cms):4.993946731234867

Distance(cms):4.799301919720768

Distance(cms):5.064456721915286

Distance(cms):4.804892253931276

Distance(cms):5.064456721915286

Distance(cms):4.934210526315789

Distance(cms):5.130597014925373

Distance(cms):4.919499105545617

Distance(cms):5.143391521197008

Distance(cms):4.919499105545617

Distance(cms):5.130597014925373

Distance(cms):4.996971532404603

Distance(cms):4.996971532404603

Distance(cms):5.10519801980198

Distance(cms):5.027422303473492

Distance(cms):5.039706780696396

Distance(cms):5.006067961165049

Distance(cms):5.121042830540037

Distance(cms):5.0

Distance(cms):5.048959608323133

Distance(cms):4.984894259818731

Distance(cms):5.165936130244208

Distance(cms):5.012150668286756

Distance(cms):5.089450956199877

Distance(cms):4.966887417218543

Distance(cms):5.0831792975970425

Distance(cms):4.858657243816254

Distance(cms):5.04278728606357

Distance(cms):4.730504587155964

Distance(cms):4.978877489438744

Distance(cms):4.650507328